# Intro to NumPy

In [ ]:
import numpy as np

## Advanced Indexing Techniques

### Boolean Indexing

As we've seen before, array comparisons result in an array of True / False values that indicate the result of that comparison for each element.

In [ ]:
# Create an array
arr = np.array([10, 15, 20, 25, 30])

# Create a boolean mask (True/False array)
mask = arr > 20
print("Mask:", mask)

That mask can be used to select elements of the original array (or any of the same shape).

In [ ]:
# Use boolean indexing to select elements
filtered = arr[mask]
print("Values > 20:", filtered)

Since the original comparison is just an expression, you can use it directly in the index operator:

In [ ]:
# new array with all values of arr less than 20
arr[arr < 20]

This can be mixed with traditional indexing and slicing, as shown in the examples below.

In [ ]:
arr_2d = np.array([[1, 2, 3, 4],
                   [5, 6, 7, 8],
                   [9, 10, 11, 12]])

# select all rows where the first element is greater than 3
row_mask = arr_2d[:, 0] > 3

print(row_mask)

In [ ]:
# with indexing: column 2 of the previous result
idx = arr_2d[row_mask, 2]
print("Column 2 for selected rows:\n", idx)

# with slicing: second and third columns of the masked rows
slice = arr_2d[row_mask, 1:3]
print("\nCols 1:3 for selected rows:\n", slice)

If we want the opposite, the tilde character (`~`) negates the condition. Here `~row_mask` will give us the rows not previously selected.

In [ ]:
~row_mask

In [ ]:
# indexing example, negated
idx = arr_2d[~row_mask, 2]
print("Column 2 for selected row:\n", idx)

# slicing example, negated
slice = arr_2d[~row_mask, 1:3]
print("\nCols 1:3 for selected row:\n", slice)

These operations can be expanded further by combining conditionals using NumPy's Boolean arithmetic operators `&` for and, and `|` for or. For example, the following statement would create a Boolean array containing the value True for every element of `names` that was equal to "Bob" or "Will". All other elements would be False.

`mask = (names == "Bob") | (names == "Will")`

Note that the Pandas keywords `and` and `or` doe not work with NumPy Boolean arrays. You must use the symbols instead.

### Fancy Indexing

NumPy provides one last indexing trick, and it is fancy. Anywhere we've used an integer to index or slice an array, you can use a list of integers instead.

In [ ]:
arr = np.zeros((8, 4))

for i in range(len(arr)):
    arr[i] = i

arr

To select a subset of rows, use a list (or `ndarray`) of integers specifying the desired order.

In [ ]:
arr[[4, 3, 0, 6]]

When assigning the result to a new variable, fancy indexing always creates a new array. When used to assign values, the indexed values will be modified. We will explore fancy indexing in more detail as required.

Together, these tools provide a powerful way to operate on specific values in an array based on conditions and location in the data. All these benefits convey to Pandas.

## Universal Functions

A universal function, or `ufunc`, is a function that performs element-wise operations on data in an `ndarray`. To get the benefit of vectorized operations (speed, memory efficiency), you must use them instead of the base Python equivalents.

We've covered several already, but many others exist. See the [NumPy documentation](https://numpy.org/doc/stable/reference/ufuncs.html) for a complete list and additional details. There, the available `ufuncs` are grouped as follows:

- Math Operations
- Trigonometric Functions
- Bit-twiddling Functions (not class relevant)
- Comparison Functions
- Floating Functions


## Broadcasting

## Array-Oriented Programming